In [74]:
#DS1ex4_10720129張書維
class Queue :    # 存放資訊的Node
    def __init__(self,OID,Timeout,Arrival,Duration,Depart,ArriCPU) :
        self.OID = OID
        self.Timeout = Timeout
        self.Arrival = Arrival
        self.Duration = Duration 
        self.Depart = Depart
        self.ArriCPU = ArriCPU
        self.next = None
        return

In [75]:
class QueueList :  # QueueList
    def __init__(self) :
        self.size = 0 
        self.head = None 
        self.tail = None
        return
    def Enqueue(self,Node) :  # 新增
        self.size += 1
        if self.head == None :
            self.head = self.tail = Node 
        else :
            self.tail.next = Node 
            self.tail = Node
        return
    def Dequeue(self) :     # 移除
        if self.head == None :
            print("Empty QUEUE !!")
            return 
        if  self.head.next != None :
            self.head =self.head.next 
        else :
            self.head = None 
        self.size-=1 
        return

In [76]:
class CPU :  # CPU 
    def __init__(self) :
        self.Abort = QueueList()  # 取消清單
        self.Done = QueueList()  #完成清單
        self.runstate = QueueList()  #正在執行的job
        self.waitstate = QueueList() #等待job
        self.delay = 0 
        self.successrate = 0
        self.data = 0
        self.time = 0 
        return

In [77]:
def Readfile(fileName):  # 讀黨
    List = QueueList()
    fileName = 'sorted' +fileName +'.txt'
    print("file open")
    with open(fileName,'r') as f : 
         data=f.readlines()
    i = 1 
    num = '0'
    while i < len(data) :
        j = 0 
        ch = data[i][j]  
        
        while ch != '\t' :
            num += ch  
            j+=1 
            ch = data[i][j]  
        OID = int (num)
        num = '0'
        j+=1
        ch = data[i][j]
        while ch != '\t' :
            num += ch  
            j+=1 
            ch = data[i][j]  
        Arrival = int (num)
        j+=1
        num = '0'
        ch = data[i][j]
        while ch != '\t' :
            num += ch  
            j+=1 
            ch = data[i][j]  
        Duration = int (num) 
        j+=1
        num = '0'
        ch = data[i][j]
        while ch != '\n' :
            num += ch  
            j+=1 
            ch = data[i][j]  
        Timeout = int (num)
        i+=1 
        num = '0'
        Node = Queue(OID,Timeout,Arrival,Duration,0,0)
        List.Enqueue(Node)

    return List

In [78]:
def Writefile(cpu,fileName) :   # 寫檔
    fileName = "output" + fileName + ".txt" 
    f =open(fileName, "w") 
    print("\t[Abort job]\n\tOID\tAbort\tDelay",file = f)
    walk = cpu.Abort.head
    i = 1
    while walk != None :
        print("[",i,"]\t",walk.OID,'\t',walk.Depart,'\t',walk.Depart-walk.Arrival, file = f)
        cpu.delay += walk.Depart-walk.Arrival
        walk= walk.next
        i+=1
    i = 1
    print("\t[Done job]\t\n\tOID\tDepartrue\tDelay",file =f)
    walk = cpu.Done.head
    while walk != None :
        print("[",i,"]\t",walk.OID,'\t',walk.Depart,'\t',walk.Depart-(walk.Arrival+walk.Duration), file = f)
        cpu.delay += walk.Depart-(walk.Arrival+walk.Duration)
        walk = walk.next
        i+=1   
    print("[Average Delay]\t",round(cpu.delay/cpu.data,2),"ms",file =f)
    print("[Success Rate]\t",round(cpu.Done.size/cpu.data*100,2),"%",file = f)
    f.close()
    print("See ",fileName,'\n')
    

In [79]:
def Simulate() :     # 模擬CPU排程
    cpu = CPU()    # CPU
    print("Input the fileName >>>>")
    
    fileName = input() 
    List = Readfile(fileName) # 所有工作名單 依照Arrival排序
    timeout = False   # 紀錄當前時刻 是否有 timeout 的情況
    finish = False    # 紀錄當前時刻 是否有 finish 的情況
    Duration = -1     # 當前工作剩餘時間
    print( "The Simulation is running...\n")
    while List.size != 0 or cpu.runstate.size != 0 :              # 還有job就繼續
        if cpu.runstate.size != 0 :                                    # 若runstate有工作 就檢視是否有timeout
            if Duration == 0 or cpu.time >= cpu.runstate.head.Timeout :# 或是 finish 的情況
                if Duration == 0 :
                    if cpu.time <= cpu.runstate.head.Timeout :
                        finish = 1
                    else :
                        timeout = 1
                if cpu.time == cpu.runstate.head.Timeout :
                    if Duration > 0 :
                        timeout = 1
                if cpu.time > cpu.runstate.head.Timeout :
                    timeout = 1 
                cpu.runstate.head.Depart = cpu.time
        next = False  # 有 finishe 或 timeout 的情況
        if timeout or finish :              # 根據情況將工作放置於完成清單或是取消清單
            New = cpu.runstate.head
            cpu.runstate.Dequeue()
            New.next = None
            if finish :
                cpu.Done.Enqueue(New)
                finish = False
            elif timeout :
                cpu.Abort.Enqueue(New)
                timeout = False
            next = True
        if List.size != 0:              # 還有工作
            if cpu.time >= List.head.Arrival :  # 工作抵達
                if cpu.runstate.size == 0 :       # CPU閒置
                    if cpu.waitstate.size == 0 :  # 等待區沒人 直接從LIST 取得工作
                        New = List.head 
                        List.Dequeue()
                        cpu.data+=1
                    else :                      # 等待區有人 從等待區取得工作
                        New = cpu.waitstate.head
                        cpu.waitstate.Dequeue()
                    New.next = None
                    cpu.runstate.Enqueue(New)
                    cpu.runstate.head.ArriCPU = cpu.time
                    Duration = cpu.runstate.head.Duration
                else :                   # CPU忙碌中
                    cpu.data +=1
                    if cpu.waitstate.size != 3 :  # 等待區還有空位 將抵達工作放入等待區
                        New = List.head
                        List.Dequeue()
                        New.next = None
                        cpu.waitstate.Enqueue(New)
                        
                    else :                      # 等待區無空位 將抵達工作丟入取消清單
                        List.head.Depart = cpu.time
                        New = List.head
                        List.Dequeue()
                        New.next = None
                        cpu.Abort.Enqueue(New)
        if cpu.waitstate.size != 0 :         # 等待區有工作
            if cpu.runstate.size == 0 :     # CPU閒置 取得工作
                New = cpu.waitstate.head 
                cpu.waitstate.Dequeue()
                New.next = None
                cpu.runstate.Enqueue(New)
                cpu.runstate.head.ArriCPU = cpu.time
                Duration = cpu.runstate.head.Duration
        walk = False  # 紀錄時間在沒有finish或timeout的情況下是否該前進
        if List.size != 0 :
            if List.head.Arrival > cpu.time :  
                walk = True
        if List.size == 0 :
            walk = True
        if cpu.data == 0 :   # 還沒有任何工作抵達 時間持續跑
            cpu.time += 1
        else :              # 已有工作抵達
            if next : # 當前時刻有finish 或 timeout的情況
                if cpu.waitstate.size == 0 :   # 等待區無工作
                    if List.size != 0 :
                        if cpu.runstate.size == 0 :
                            if List.head.Arrival > cpu.time :  # CPU閒置 下一個工作尚未抵達 時間持續跑
                                cpu.time+=1      
                        elif cpu.time > cpu.runstate.head.Timeout :  # CPU忙碌 工作已逾時 
                             Duration=0                         #時間停止 將當前工作剩餘時間歸零
                elif cpu.waitstate.head.Arrival > cpu.time :  # # 等待區有工作 下一個工作尚未抵達 時間持續跑
                    cpu.time+=1
                    Duration-=1
            elif walk : # 當前時刻沒有finish 或 timeout的情況
                cpu.time +=1
                Duration-=1
        
    Writefile(cpu,fileName)          
                        
        
        
        
        


In [80]:


 def main() :
    print("**** Simulate QUEUES BY SQF *****\n****----0---- QUIT   ***********\n****---1---Simulate one QUEUE ****\n")
    cmd = input("Input a command(0, 1):")
    while cmd != '0' :
        if cmd == '1' :
            Simulate()
        cmd = input("Input a command(0, 1):") 
        


In [81]:
main()

**** Simulate QUEUES BY SQF *****
****----0---- QUIT   ***********
****---1---Simulate one QUEUE ****

Input a command(0, 1):1
Input the fileName >>>>
401
file open
The Simulation is running...

See  output401.txt 

Input a command(0, 1):1
Input the fileName >>>>
402
file open
The Simulation is running...

See  output402.txt 

Input a command(0, 1):0
